In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm
from scipy.stats import chi2
from scipy.special import stdtr
from scipy.stats import t
from matplotlib.widgets import Slider, Button
%matplotlib notebook

We simplify the experiment in the video to the following setup.  We have an experiment where we are showing the salmon 15 images.  For a fixed voxel then we can observe 15 values, one for each image, of the voxel in question.  We assume that we get an activation signal $X_i\sim\mathcal{N}(\mu, \sigma^2)$ for $i= 1,\dots,n, n=15$  Then our hypotheses are

$$
H_0: \mu=0
$$
$$
H_1: \mu \ne 0
$$

So we have a small number of samples which points us to looking at a T-test.  We want to test the hypothesis at a level of $\alpha = 0.1\%$  What does it mean to run the test at this level?  It means that at most $0.1\%$ of the tests will reject the null hypothesis in error.  So if, as in our case, we are running this test repeatedly, in our case $8,064$, then we should expect to reject the null in error about 8 times.

### Multiple Testing

So what went wrong with the analysis about the salmon?  We made a mistake by trying to make a conclusion about "all the voxels" at once.  If we have a lot of hypotheses we are testing at once, we are _certain_ that we are going to make some mistakes.  There are two solutions to this.

#### Control the Family Wise Error Rate

We can try to control not a single hypothesis but try to control what is called the _Family Wise Error Rate_ or FWER.  Let's assume that we have $N$ tests to run at once.  To do that we try to find $C_1, \dots , C_N$ such that

$$
\mathbb{P}_{\mu_i=0}\left( \bigcup_{i=1}^N\lbrace |T_i| \gt C_i\rbrace\right) \le \alpha
$$

Let's unpack this a bit.  In each of our experiments we want to find a control for the probability of falsely rejectinng the null.  That is the role that the $C_i$ play.  In order to bound the growith of all of those errors we impose the condition that the total probabiliyt of at least one test falsely rejecting is at most $\alpha$.  That is why we take the union of all the rejection regions.  But this is a _very_ conservative bound.  If we have that $N$ is very large, then we will in effect be asking to reduce the level for an individual test to a truly tiny amount.  Remember we are lookin at bounding the probability that just one test falsely rejects out of all of our tests, and there could be thousands of tests we are running.  There is an alternative

#### Control the False Discovery Rate

Benjaminin and Hochberg proposed a different approach.  They proposed to control the expected proportion of false discoveries (incorectly rejecting the null hypothesis). To do that we again find some bounds $C_1,\dots, C_N$ such that

$$
\text{FDR}:=\mathbb{E}\left[ \frac{\text{#}\lbrace i:|T_i|\gt C_i \text{and} \mu = 0 \rbrace}{\text{#}\lbrace i:|T_i|\gt C_i\rbrace}\right]=\mathbb{E}_{\mu = 0}\left[\frac{\text{# of false discoveries}}{\text{# of discoveries}}\right]\lt \alpha
$$

So how do we come up with the $C_i$?  We can use the p-values for each of the tests.

$$
P_i = \mathbb{P}_{\mu = 0}\left[ |T| \gt |t_i^{\text{obs}}| \right]
$$

where $t_i^{\text{obs}}$ is the observed value of the test statistic for the $i$th test, and $T\sim t_{n-1}$.

For a series of tests in which the $i$th test uses a null hypothesis of $H_0^i$, let the totoal number of each type of outcome be defined per this table:

|             |Do not reject $H_0^i$ (non significant)|Reject $H_0^i$(significant)       |      |
|:-           |:-                                     |:-                                |:-    |
|$H_0^i$ true |$A$ (Number of true non significant)   |$V$ (Number of type I errors)     | $m_0$|
|$H_0^i$ false|$C$ (Number of type II errors)         |$D$ (Number of true significance) | $m_1$|
|             |$N_0$ (Number of non-significance)     |$N_1$ (Number of significance)    |      |

Then the family-wise error rate (FWER) is the probability of making at least one false discovery, or type I error:

$$
\text{FWER} = \mathbb{P}(V\gt 1)
$$

where $V$ is the total number of type 1 errors. That is $V=\sum_{i=1}^{m_0}\Psi$ where $\lbrace \Psi_i\rbrace$ is the set of $m_0$ test for which $H_0$ is true.  This is the most conservative aproach.  So in cases where the consequences of false discovery are high, this is the approach to use.  However, we pay the price of not being able to discover anything.  For example suppose we have a paird test where there are 1000 participants and we are measuring 100 variables, and the treatment itself is just water.  Then

$$
\text{FWER}=\mathbb{P}(V\ge 1)= 1 - \mathbb{P}(V=0)= 1-(1-\alpha)^m\approx 1
$$

when $m$ is large.

The False Discovery Rate is the total number of false discoveries $V$ divided by the number of discoveries made $N_1$.  Since a discovery can't be false and not be a discovery we have that $V\le N_1$ and thus $0\le \text{FDR}\le1$ under the condition that $\text{FDR}=0$ if $N_1=0$.  Thus we have that

$$
\text{FDR}=\mathbb{E}\left[\frac{V}{N_1}\right]
$$

If we compare the two approaches we see that the FDR method has more power than FWER in the sense that using FRD is lest strict a condition than FWER.  How do we see this?  Consider a trivial scenario where all of our hypotheses are true.  In this case any rejected null must be false.  Hence $V=N_1$ and so

$$
\mathbb{E}\left[ \frac{V}{N_1}\right]=\mathbb{P}(V\ge 1)
$$

On the other hand suppose that some null hyptheses are false.  Now if $N_1\gt0$ we only know that $V/N_1\le 1$.  We define an indicator variable $\mathbb{1}(V\ge 1)$ which takes a value $1$ when $V\ge 1$. Then

$$
\frac{V}{N_1}\le \mathbb{1}(V\ge 1) =1 \text{ when } V \ge 1
$$
and
$$
\frac{V}{N_1}= \mathbb{1}(V\ge 1) \text{ when } V = 0
$$

Taking expectations on both sides we see that

$$
\mathbb{E}\left[ \frac{V}{N_1} \right] \le \mathbb{E}[\mathbb{1}(V\ge 1)] = \mathbb{P}(V\ge 1)
$$

and thus $\text{FDR}\le\text{FWER}$.  Since the $\text{FDR}$ is smaller, it is easier to control and still make discoveries.  We do not have to impose as tight a correcton factor to $\text{FDR}$ as we do to $\text{FWER}$ to get under a threshold $\alpha$.

### The Bonferroni Correction

The classical way to control the FWER is to use what is called the Bonferroni Correction.  It is simply to insteda of rejecting each test at level $\alpha$ we reject at a (_much_) smaller level $\alpha/N$, where $N$ is the number of tests we are considering.  So what is this doing? It is shrinking the rejection region such that we will only reject a test if the p-value is less than $\alpha/N$.  If $N$ is large we see that this can be a very small number.  This is often overly conservative, but lets make sure this does actualy do what we want.


$$
\text{FWER}=\mathbb{P}_{\mu_i =0} \left( \bigcup_{i=1}^N\lbrace P_i < \frac{\alpha}{N} \rbrace \right)=
\mathbb{P}_{\mu_i = 0}\left( \bigcup_{i=1}^{N} \lbrace |T_i| \gt q_{\frac{\alpha}{2N}}^{t_n-1} \rbrace \right)
$$

where this is just writing down the definitions and choosing the same $C_i$ for each test.  Now using the fact that $\mathbb{P}(A\cup B)=\mathbb{P}(A) + \mathbb{P}(B)$ we can rewrite this as

$$
\le \sum_{i=1}^{N}\mathbb{P}_i(|T_i|\gt q_{\frac{\alpha}{2N}}^{t_n-1})=\sum_{i=1}^{N}\frac{\alpha}{N}=\alpha
$$

### The Benjamini-Hochberg Correction

It is a sophisticated method that was developed at the end of the 20th century.  First of all compute the p-value for each of the tests and then order the p-values.  That is we have $P_{(1)}\lt P_{(2)}\lt P_{(3)}\lt\dots\lt P_{(N)}$.  So $P{(i)}$ is the $i$th smallest p-value of all of the tests we have performed.  The idea behind the correction is to reject all the tests $(i)$ where $i\lt i_{\text{max}}$.  Now the question is, how do we choose that number?  It should probably depend on $N$, and it should probably depend on $\alpha$, but what is the magic formula?

We start with the Bonferroni correction an add in the factor $i$

$$
i\frac{\alpha}{N}
$$

What will this do?  First notice that the test with the largest p-value is subject to the level $\alpha$.  At the other end the test with the smallest p-value is subject to a level of $\alpha/N$ or the Bonferroni correction.  If $N$ is very large, then it is likely that we will have a false discovery. As we move up our list of tests with increasing p-values we will subject those tests to slightly larger levels weighted by their position in the list.  As we move up the list we will get to a point where the test where the p-value for that test is less than it's position in the list times the Bonferroni Correction.  For that index $i$ we drop all of the tests below that level.  Here is what that looks like in notation.

$$
i_{\text{max}}:=\text{max}\lbrace i: P_{(i)} \lt i\frac{\alpha}{N}\rbrace
$$

What Benjamini and Hochberg showed was that if the p-values are independent then if we use that method then we can control the FDR by $\alpha$

$$
\text{FDR}\le \alpha
$$

As for the condition of independence, in lot's of situations where this would be useful we don't have true independece.  For instance in the salmon example neighboring voxels would tend to activate together.  Genes tend to have complex interrelatednes etc.  The literature has lots of variations on this method that address these concerns.